# Invoke Bedrock model for text generation using zero-shot prompt

> *This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio*

## Introduction
You are Bob a Customer Service Manager at AnyCompany and some of your customers are not happy with the customer service and are providing negative feedbacks on the service provided by customer support engineers. Now, you would like to respond to those customers humbly aplogizing for the poor service and regain trust. You need the help of an LLM to generate a bulk of emails for you which are human friendly and personalized to the customer's sentiment from previous email correspondence.

## Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import os
import sys
import boto3
import botocore
import ipywidgets as widgets

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from labutils import bedrock, model_formatter, tools

In [3]:
bedrock_runtime_client, bedrock_control_client = bedrock.get_bedrock_clients()

Create new client
  Using region: None
  Using profile: default
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)
Create new client
  Using region: None
  Using profile: default
boto3 Bedrock client successfully created!
bedrock(https://bedrock.us-east-1.amazonaws.com)


## Create the prompt

In [4]:
# Define a simple prompt
user_input = """
Write an email from Bob, Customer Service Manager, to the customer "John Doe" 
who provided negative feedback on the service provided by our customer support 
engineer"""

## Select the model

In [5]:
dropdown = tools.model_selection(bedrock_control_client)
display(dropdown)

Dropdown(description='Select LLM:', layout=Layout(width='300px'), options={'Claude': 'anthropic.claude-v2:1', …

## Build the JSON request payload

In [17]:
modelId = dropdown.value
body = model_formatter.model_body_builder(modelid  = modelId,
                                           user_input  = user_input,                                             
                                           temperature = 0.9,
                                           topP = 0.99,
                                           topK = 250,
                                           maxTokenCount = 512,
                                           stopSequences = [])

In [18]:
print(body)

{"inputText": "Command: \nWrite an email from Bob, Customer Service Manager, to the customer \"John Doe\" \nwho provided negative feedback on the service provided by our customer support \nengineer", "textGenerationConfig": {"maxTokenCount": 512, "stopSequences": [], "temperature": 0.9, "topP": 0.99}}


## Invoke the LLM

In [13]:
try:

    response = bedrock_runtime_client.invoke_model(body = body, modelId = modelId)

except botocore.exceptions.ClientError as error:
    
    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
        
    else:
        raise error


In [14]:
print(response)

{'ResponseMetadata': {'RequestId': '91d4fb7d-963a-49ef-806f-7be411bdfcbe', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Wed, 03 Jul 2024 13:36:23 GMT', 'content-type': 'application/json', 'content-length': '947', 'connection': 'keep-alive', 'x-amzn-requestid': '91d4fb7d-963a-49ef-806f-7be411bdfcbe', 'x-amzn-bedrock-invocation-latency': '4791', 'x-amzn-bedrock-output-token-count': '168', 'x-amzn-bedrock-input-token-count': '37'}, 'RetryAttempts': 0}, 'contentType': 'application/json', 'body': <botocore.response.StreamingBody object at 0x000001FF11A18DC0>}


## Print the response

In [15]:
print(model_formatter.model_response_parser(modelId, response))

, James
Here is a draft email to a customer who has provided negative feedback:

Dear [Insert name],

I am writing to address your negative feedback regarding the service provided by our customer support engineer, James.

At [Insert company name], we strive to deliver the best possible customer experience. I am truly sorry to hear that James did not meet your expectations. We take customer feedback very seriously and will investigate this matter thoroughly.

We appreciate your feedback and value your business. I understand that negative feedback can be frustrating, but please believe that we will use it to improve our services.

I would like to invite you to contact me directly at your convenience. I will be happy to address any concerns you may have and provide a solution that meets your needs.

Sincerely,
[Insert name]


## Streaming the response

In [19]:
output = []
try:
    
    response = bedrock_runtime_client.invoke_model_with_response_stream(body=body, modelId=modelId)
    stream = response.get('body')
    
    i = 1
    if stream:
        for event in stream:
            chunk = event.get('chunk')
            if chunk:
                chunk_obj = json.loads(chunk.get('bytes').decode())
                text = chunk_obj['outputText']
                output.append(text)
                print(f'\t\t\x1b[31m**Chunk {i}**\x1b[0m\n{text}\n')
                i+=1
            
except botocore.exceptions.ClientError as error:
    
    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
        
    else:
        raise error

		**Chunk 1**

Here is a draft email to a customer who provided negative feedback:

Dear John Doe,

I was recently notified of a negative customer feedback regarding our customer support engineer.

I would like to apologize for the negative

		**Chunk 2**
 experience you had. We strive to provide excellent service to all of our customers, and I apologize if we fell short in your case.

Sincerely,
[Insert name]



### Complete output

In [20]:
print('\t\t\x1b[31m**COMPLETE OUTPUT**\x1b[0m\n')
complete_output = ''.join(output)
print(complete_output)

		**COMPLETE OUTPUT**


Here is a draft email to a customer who provided negative feedback:

Dear John Doe,

I was recently notified of a negative customer feedback regarding our customer support engineer.

I would like to apologize for the negative experience you had. We strive to provide excellent service to all of our customers, and I apologize if we fell short in your case.

Sincerely,
[Insert name]
